In [21]:
import numpy as np

In [56]:
# Working example for my blog post at:
# https://danijar.github.io/structuring-your-tensorflow-models
import functools
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


def doublewrap(function):
    """
    A decorator decorator, allowing to use the decorator to be used without
    parentheses if not arguments are provided. All arguments must be optional.
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator


@doublewrap
def define_scope(function, scope=None, *args, **kwargs):
    """
    A decorator for functions that define TensorFlow operations. The wrapped
    function will only be executed once. Subsequent calls to it will directly
    return the result so that operations are added to the graph only once.
    The operations added by the function live within a tf.variable_scope(). If
    this decorator is used with arguments, they will be forwarded to the
    variable scope. The scope name defaults to the name of the wrapped
    function.
    """
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            #with tf.variable_scope(name, *args, **kwargs):
            setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return decorator

################################################################################################
################################################################################################
################################################################################################

class Model:

    def __init__(self, input_dim, embedding_dim, encoder_layers,
        transfer_function=tf.nn.relu, loss_function=tf.metrics.mean_squared_error, regularizer=tf.contrib.layers.l1_regularizer, 
        lambda_scalar=.001, opt_method=tf.train.RMSPropOptimizer
    ):
        self._input_dim = input_dim
        self._embedding_dim = embedding_dim
        self._encoder_layers = encoder_layers
        self._transfer_function = transfer_function
        self._loss_fucntion = loss_function
        self._regularizer = regularizer(scale=lambda_scalar)
        self._lambda_scalar = lambda_scalar
        self._opt_method = opt_method

        with tf.name_scope('input'):
            self._X = tf.placeholder(tf.float32, [None, input_dim])
        with tf.name_scope('label'):
            self._y = tf.placeholder(tf.float32, [None, input_dim])

        self.prediction
        self.optimize
        self.error



#     @define_scope(initializer=tf.contrib.slim.xavier_initializer(), regularizer=tf.contrib.layers.l1_regularizer(scale=.001))
#     def prediction(self):
#         x = self._X
#         prev_layer_dim = self._input_dim
        
#         with tf.name_scope("encoder"):
#             for i, layer_dim in enumerate(self._encoder_layers):
#                 w_str = 'w_{}_encoder'.format(i)
#                 x_str = 'x_{}_encoder'.format(i)
#                 w = tf.get_variable(w_str, shape=[prev_layer_dim, layer_dim])
#                 x = self._transfer_function(tf.matmul(x,w))
#                 #self._acts[x_str] = x
#                 prev_layer_dim = layer_dim
#             w_str = 'w_{}_middle'.format(len(self._encoder_layers))
#             w = tf.get_variable(w_str, shape=[prev_layer_dim, self._embedding_dim])

#         with tf.name_scope("middle layer"):
#             x = tf.matmul(x, w, name='middle code')
#             prev_layer_dim = self._embedding_dim

#         with tf.name_scope("decoder"):
#             for i, layer_dim in enumerate(reversed(self._encoder_layers)):
#                 w_str = 'w_{}_decoder'.format(i)
#                 x_str = 'x_{}_decoder'.format(i)
#                 w = tf.get_variable(w_str, shape=[prev_layer_dim, layer_dim])
#                 x = self._transfer_function(tf.matmul(x,w))
#                 #self._acts[x_str] = x
#                 prev_layer_dim = layer_dim
#             w_str = 'w_{}_output'.format(len(self._encoder_layers))
#             w = tf.get_variable(w_str, shape=[prev_layer_dim, self._input_dim])

#         with tf.name_scope("output"):
#             x = tf.matmul(x, w, name='output code')
#         return x 
    
    @define_scope(initializer=tf.contrib.slim.xavier_initializer(), regularizer=tf.contrib.layers.l1_regularizer(scale=.001), reuse=None)
    def prediction(self):
        x0 = self._X
        #with tf.name_scope("encoder"):

        w1 = tf.get_variable('w1', shape=[self._input_dim, self._encoder_layers[0]])
        x1 = self._transfer_function(tf.matmul(x0, w1))

        w2 = tf.get_variable('w2', shape=[self._encoder_layers[0], self._encoder_layers[1]])
        x2 = self._transfer_function(tf.matmul(x1, w2))

        w3 = tf.get_variable('w3', shape=[self._encoder_layers[1], self._encoder_layers[2]])
        x3 = self._transfer_function(tf.matmul(x2, w3))

        w4 = tf.get_variable('w4', shape=[self._encoder_layers[2], self._embedding_dim])
        x4 = tf.matmul(x3, w4)
      #with tf.name_scope("decoder"):

        w5 = tf.get_variable('w5', shape=[self._embedding_dim, self._encoder_layers[2]])
        x5 = self._transfer_function(tf.matmul(x4, w5))

        w6 = tf.get_variable('w6', shape=[self._encoder_layers[2], self._encoder_layers[1]])
        x6 = self._transfer_function(tf.matmul(x5, w6))

        w7 = tf.get_variable('w7', shape=[self._encoder_layers[1], self._encoder_layers[0]])
        x7 = self._transfer_function(tf.matmul(x6, w7))

        w8 = tf.get_variable('w8', shape=[self._encoder_layers[0], self._input_dim])
        x8 = tf.matmul(x7, w8)

        return x 


    @define_scope
    def loss(self):
        error_part = self._loss_fucntion(self._y, self.prediction)
        reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        loss = error_part +  sum(reg_losses)
        return loss 

    @define_scope
    def optimize(self):
        optimizer = self._opt_method(0.03)
        return optimizer.minimize(self.loss)

In [32]:
sess = tf.Session()
writer = tf.summary.FileWriter('./some_graph')

In [43]:
sess.close()

In [36]:
-


In [37]:
X, y = generate_data(500, 10)

In [39]:
y.shape

(499, 10)

In [57]:
instance = Model(input_dim = 10, embedding_dim = 4, encoder_layers=[8, 8, 6])

ValueError: Variable w1 already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\Program Files\Anaconda2\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 1264, in __init__
    self._traceback = _extract_stack()
  File "C:\Program Files\Anaconda2\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 2395, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\Program Files\Anaconda2\envs\tensorflow\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 763, in apply_op
    op_def=op_def)
